<a href="https://colab.research.google.com/github/HANE48/colab_Deep_learining/blob/main/CBOW_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class CBOW:
    def __init__(self, vocab_size, embedding_dim):
        self.W1 = np.random.randn(embedding_dim, vocab_size) * 0.01
        self.W2 = np.random.randn(vocab_size, embedding_dim) * 0.01
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

    def forward(self, x):
        h = np.dot(self.W1, x)
        u = np.dot(self.W2, h)
        y_pred = softmax(u)
        return y_pred, h

    def backward(self, x, y_true, h, y_pred, learning_rate):
        dl_du = y_pred - y_true
        dl_dw2 = np.outer(dl_du, h)
        dl_dh = np.dot(self.W2.T, dl_du)
        dl_dw1 = np.outer(dl_dh, x)

        self.W1 -= learning_rate * dl_dw1
        self.W2 -= learning_rate * dl_dw2

    def train(self, data, word_to_index, context_size, epochs, learning_rate):
        for epoch in range(epochs):
            loss = 0
            for context, target in data:
                x = np.mean([self.word_to_one_hot(word_to_index[word]) for word in context], axis=0)
                y_true = self.word_to_one_hot(word_to_index[target])
                y_pred, h = self.forward(x)
                self.backward(x, y_true, h, y_pred, learning_rate)
                loss -= np.log(y_pred[word_to_index[target]])
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}, Loss: {loss}")

    def word_to_one_hot(self, word_idx):
        x = np.zeros(self.vocab_size)
        x[word_idx] = 1
        return x

# 코퍼스와 사전 처리
corpus = ["apple banana apple", "banana orange fruit", "orange banana apple", "dog cat animal", "cat monkey animal"]
words = set(" ".join(corpus).split())
word_to_index = {word: i for i, word in enumerate(words)}

# 모델 초기화 및 학습
vocab_size = len(word_to_index)
embedding_dim = 10
model = CBOW(vocab_size, embedding_dim)
data = [(["banana", "apple"], "apple"), (["banana", "orange"], "fruit"), (["orange", "apple"], "banana"), (["dog", "animal"], "cat"), (["cat", "animal"], "monkey")]
model.train(data, word_to_index, context_size=2, epochs=100, learning_rate=0.01)


Epoch 10, Loss: 10.396349157818857
Epoch 20, Loss: 10.395911270354915
Epoch 30, Loss: 10.395447294101121
Epoch 40, Loss: 10.394947278991904
Epoch 50, Loss: 10.394400326663707
Epoch 60, Loss: 10.393794288443699
Epoch 70, Loss: 10.393115425325334
Epoch 80, Loss: 10.392348019376048
Epoch 90, Loss: 10.39147392453327
Epoch 100, Loss: 10.39047204290058
